In [8]:
#init_notebook_mode(connected=True)
%matplotlib inline
import matplotlib, plotly, fileinput
import matplotlib.pyplot as plt
import pandas as pd 
from collections import Counter

from plotly.offline import plot, iplot, init_notebook_mode
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [9]:
dates = []
for line in fileinput.input('dates.txt'):
    line = line.rstrip()
    
    #ignore lines starting with # or ending with ?
    if line[0] == '#' or line[-1] == '?':
        continue
    
    _, date = line.split('|') # split lines
    date = date.replace('', '') # remove spaces
    dates.append(pd.Timestamp(date))

In [10]:
colors = ['#eebcbc', '#eedeab', '#c8ecb5', '#bcc2f2', '#e8b6ee', '#b25a86', '#546e85']

#all data together
d = Counter([t.weekday_name for t in dates])
labels = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
all_values = [d[l] for l in labels]

empty_trace = go.Pie(labels=[], values=[],
               hoverinfo='label+percent', textinfo='value', 
               textfont=dict(size=20), name='None',
               marker=dict(colors=colors, 
                           line=dict(color='#000000', width=2)))

main_trace = go.Pie(labels=labels, values=all_values,
               hoverinfo='label+percent', textinfo='value', 
               textfont=dict(size=20), name='Total',
               marker=dict(colors=colors, 
                           line=dict(color='#000000', width=2)))

print('All', all_values, labels)

All [10, 12, 13, 11, 12, 9, 91] ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']


In [11]:
#per decade
weekday_per_decade = {}

for t in dates:
    decade = (t.year//10)*10
    #print(decade, t.weekday_name)
    if decade not in weekday_per_decade:
        weekday_per_decade[decade] = []
    weekday_per_decade[decade].append(t.weekday_name)

#traces
traces = [empty_trace, main_trace]
for decade in weekday_per_decade:
    c = Counter(weekday_per_decade[decade])
    weekday_per_decade[decade] = c
    
    current_values = []
    current_labels = []
    current_colors = []
    for j, l in enumerate(labels):
        if c[l] > 0:
            current_labels.append(l)
            current_values.append(c[l])
            current_colors.append(colors[j])
    
    g = go.Pie(labels=current_labels, values=current_values,
               hoverinfo='label+percent', textinfo='value', 
               textfont=dict(size=20), name=str(decade),
               marker=dict(colors=current_colors, 
                           line=dict(color='#000000', width=2)))
    traces.append(g)
    print(decade, current_values, current_labels)
    
#traces

1830 [1, 3, 2] ['Monday', 'Wednesday', 'Friday']
1840 [1, 2, 2, 1] ['Monday', 'Friday', 'Saturday', 'Sunday']
1850 [1, 1, 2, 2] ['Monday', 'Wednesday', 'Saturday', 'Sunday']
1860 [2, 2, 1] ['Tuesday', 'Thursday', 'Friday']
1870 [1, 2, 1, 2] ['Monday', 'Thursday', 'Saturday', 'Sunday']
1880 [1, 2] ['Saturday', 'Sunday']
1890 [1, 2, 2] ['Wednesday', 'Friday', 'Sunday']
1900 [1, 1, 2] ['Monday', 'Thursday', 'Sunday']
1910 [3, 1, 1] ['Monday', 'Friday', 'Sunday']
1920 [2, 1] ['Tuesday', 'Sunday']
1930 [1, 7] ['Friday', 'Sunday']
1970 [1, 3, 2, 3] ['Monday', 'Wednesday', 'Thursday', 'Sunday']
1980 [6, 2, 4, 3, 1, 10] ['Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
2000 [1, 3, 1, 20] ['Monday', 'Wednesday', 'Saturday', 'Sunday']
1990 [2, 1, 21] ['Tuesday', 'Saturday', 'Sunday']
2010 [17] ['Sunday']


In [12]:
#button list

keys = ['None', 'Total']
keys += list(weekday_per_decade.keys())

button_list = []
for i, decade in enumerate(keys):
    bools = [False]*len(keys)
    if i == 0:
        button = dict(label = str(decade),
          method = 'update',
          args = [{'visible': bools},
          {'title': 'Select a year'}])
    else:
        bools[i] = True
        button = dict(label = str(decade),
          method = 'update',
          args = [{'visible': bools},
          {'title': 'Elections or referendums celebrated during the decade: {}'.format(decade)}])
    #add
    button_list.append(button)

In [13]:
updatemenus = list([
    dict(active=0,
         buttons=button_list,
    )
])

layout = dict(title='Select a year', showlegend=True, updatemenus=updatemenus)

fig = dict(data=traces, layout=layout)

iplot(fig, filename='styled_pie_chart')
plot(fig, filename='docs/styled_pie_chart')

/usr/local/lib/python3.6/site-packages/plotly/offline/offline.py:459: UserWarning:

Your filename `docs/styled_pie_chart` didn't end with .html. Adding .html to the end of your file.



'file:///Users/carlosvega/WeekdaySpanishElections/docs/styled_pie_chart.html'